### 설치

In [ ]:
#!pip install pandas
#!pip install plotly
#!pip install matplotlib
#!pip install librosa
# F5 (chrome reload)

### 데이터 다운로드 주소
* <https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-002>


------
### 데이터 프레임 생성 및 살펴 보기

In [ ]:
# 파일 읽어서 생성하기
import pandas as pd
df = pd.read_csv('./data/5차년도_2차.csv', encoding='cp949')
df

In [ ]:
df = df[['wav_id', '상황', '나이', '성별', '발화문']]
df


In [ ]:
df = df.copy()

In [ ]:
# 몇 개만 보기
display(df.head(5))
print('\n')
display(df.tail(5))
print('\n')
display(df.sample(n=5))
print('\n')

------
### 서브 테이블(DataFrame or Series) 추출

In [ ]:
# 한 컬럼만 분리하기.  Series
s = df['상황']
print(type(s))
s

In [ ]:
#   컬럼들 분리하기. 
i = ['상황']#, '1번 감정', '성별']
t = df[i] # t = df[['상황']]
print(type(t))
t

In [ ]:
# numpy 형태로 값 얻기
print(s.shape)
s.values

In [ ]:
# numpy 형태로 값 얻기
print(t.shape)
t.values

In [ ]:
# 특정 컬럼의 값 기준으로 테이블로 분리
g = df.groupby('상황')
for situation, df_i in g:
    print(f'상황:{situation}, ({len(df_i)}개)')
    display(df_i.tail(5))
    print('\n', '=='*50, '\n')

In [ ]:
# 유니크한 값만 추출하기 :
s = df['상황']
s.unique() #set(s.values)

In [ ]:
s.nunique() # len(set(s.values))

In [ ]:
# 특정 조건을 만족하는 row 만 골라내기. query 버전
situations = ['sadness', 'angry']
df.query('상황 in @situations') # df.query('상황 == "sadness" or 상황 == "angry"')

In [ ]:
# 특정 조건을 만족하는 row 만 골라내기. flag index 버전 
df[df['상황'] =='angry'] # df.query('상황 == "angry"')

In [ ]:
df['상황'] =='angry'

In [ ]:
type(df.상황 == 'angry')

In [ ]:
s = df.상황 == 'angry'
df[s]

------
### 그래프

In [ ]:
df['나이']

In [ ]:
df['나이'].hist()

In [ ]:
%config InlineBackend.figure_format = 'retina'
df['나이'].hist(bins=60)

In [ ]:
df['글자수'] = df['발화문'].map(len)
df

In [ ]:
df['글자수'].hist(bins=100)

In [ ]:
vc = df['상황'].value_counts()
vc

In [ ]:
import plotly.graph_objects as go
go.Figure(go.Bar(x=vc.index, y=vc.values))

In [ ]:
import plotly.express as px
px.pie(values=vc.values, names=vc.index)

------
### 데이터 검수

In [ ]:
from glob import glob
wavs = glob('./data/wavs/*.wav')
wavs

In [ ]:
from pathlib import Path
wavs  = [Path(e).stem for e in wavs]
wavs

In [ ]:
df_wav = pd.DataFrame(wavs, columns=['wav_id'])
df_wav

In [ ]:
df_wav = pd.merge(df, df_wav, how='inner', on='wav_id')
df_wav

In [ ]:
to_path = lambda e: f'./data/wavs/{e}.wav'
df_wav['path'] = df_wav['wav_id'].map(to_path)
df_wav

In [ ]:
import IPython.display as ipd

for e in df_wav.values:
    print(e)
    display(ipd.Audio(e[-1], autoplay=True))
    c = input()
    if c in ['q', 'Q', 'ㅂ']:
        break
    ipd.clear_output()

In [ ]:
import librosa
for i, e in df_wav.iterrows():
    wav, sr = librosa.load(e['path'])
    display(ipd.Audio(e['path'], autoplay=True))
    display(ipd.Audio(wav, rate=sr))
    if 'q' == input():
        break
    ipd.clear_output()

In [ ]:
print(sr, 13*sr, 14*sr)
print(wav.shape, wav.min(), wav.max(), wav.mean(), wav.std())
print(wav)

In [ ]:
ipd.Audio(wav, rate=sr, normalize=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from librosa.display import waveshow

for e in [1,10,100]:
    wav_n = (wav * e).clip(-1, 1)
    waveshow(wav_n)
    plt.show()
    display(ipd.Audio(wav_n, rate=sr, normalize=False))
    display(ipd.HTML('<hr>'))

In [ ]:
px.line(x=range(len(wav_n)), y=wav_n)